# Utilisation d'une archive Darwin Core

Pour plus de contexte voir : [README](https://github.com/OGSL-SLGO/examples/blob/main/examples/biodiversity/)

## Importation et exploration des fichiers

Les informations et les métadonnées sur l'archive Darwin Core utilisée peuvent être trouvées ici : https://catalogue.ogsl.ca/dataset/3b8c6d97-6eb8-4e8d-9869-ccb2b9bab5f3

Assurez-vous d'avoir les packages requis: 

In [1]:
library('tidyverse')

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.0      ✔ purrr   1.0.0 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.5.0 
✔ readr   2.1.3      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


L'archive utilisée comme exemple contient 4 fichiers : 
- un fichier *event core*
- un fichier *occurrence core*
- deux fichiers *measurement of facts* (emof): event et occurrence

In [2]:
url <-'https://catalogue.ogsl.ca/data/zip-rne/3b8c6d97-6eb8-4e8d-9869-ccb2b9bab5f3/zip-rne_marais-pf_event_2019-2021.csv'
event_raw <- read.csv(url)

url <-'https://catalogue.ogsl.ca/data/zip-rne/3b8c6d97-6eb8-4e8d-9869-ccb2b9bab5f3/zip-rne_marais-pf_occurrence_2019-2021.csv'
occurrence_raw <- read.csv(url)

url <-'https://catalogue.ogsl.ca/data/zip-rne/3b8c6d97-6eb8-4e8d-9869-ccb2b9bab5f3/zip-rne_marais-pf_event_emof_2019-2021.csv'
emof_event_raw <- read.csv(url)

url <-'https://catalogue.ogsl.ca/data/zip-rne/3b8c6d97-6eb8-4e8d-9869-ccb2b9bab5f3/zip-rne_marais-pf_occurrence_emof_2019-2021.csv'
emof_occurrence_raw <- read.csv(url)

head(event_raw)
head(emof_event_raw)

datasetID          countryCode eventID                eventDate       
1 CZIP-RNE-MARAIS-PF CA          CZIP-RNE-PF-2019-B1-1  2019-08-26T16:35
2 CZIP-RNE-MARAIS-PF CA          CZIP-RNE-PF-2019-B1-9  2019-08-27T08:52
3 CZIP-RNE-MARAIS-PF CA          CZIP-RNE-PF-2019-B1-15 2019-08-27T17:32
4 CZIP-RNE-MARAIS-PF CA          CZIP-RNE-PF-2019-B1-22 2019-08-28T09:42
5 CZIP-RNE-MARAIS-PF CA          CZIP-RNE-PF-2019-B1-25 2019-08-28T16:37
6 CZIP-RNE-MARAIS-PF CA          CZIP-RNE-PF-2019-B1-34 2019-08-29T11:42
  code_station num_echantillon samplingProtocol decimalLongitude
1 B1            1              Bourolle         -69.09246       
2 B1            9              Bourolle         -69.09246       
3 B1           15              Bourolle         -69.09246       
4 B1           22              Bourolle         -69.09246       
5 B1           25              Bourolle         -69.09246       
6 B1           34              Bourolle         -69.09246       
  decimalLatitude habitat               maximumElevationInMeters commentaires
1 48.64362        Marelle / Haut marais NA                                   
2 48.64362        Marelle / Haut marais NA                                   
3 48.64362        Marelle / Haut marais NA                                   
4 48.64362        Marelle / Haut marais NA                                   
5 48.64362        Marelle / Haut marais NA                                   
6 48.64362        NA                    NA

eventID               measurementID                    measurementType     
1 CZIP-RNE-PF-2019-B1-1 CZIP-RNE-PF-2019-B1-1-MESURE-01  État de la marée    
2 CZIP-RNE-PF-2019-B1-1 CZIP-RNE-PF-2019-B1-1-MESURE-49  Condition météo     
3 CZIP-RNE-PF-2019-B1-1 CZIP-RNE-PF-2019-B1-1-MESURE-97  Type de substrat    
4 CZIP-RNE-PF-2019-B1-1 CZIP-RNE-PF-2019-B1-1-MESURE-146 Condition météo     
5 CZIP-RNE-PF-2019-B1-1 CZIP-RNE-PF-2019-B1-1-MESURE-194 Vitesse du vent     
6 CZIP-RNE-PF-2019-B1-1 CZIP-RNE-PF-2019-B1-1-MESURE-242 Température de l'air
  measurementValue     measurementUnit
1 Basse et Descendante                
2 Nuageux                             
3 Argile / Vaseux                     
4 Nuageux                             
5 5                    km/h           
6 25                   °C

On veut obtenir dans un même fichier : les coordonnées où on eu lieu les occurrences, le nombre d'individus et les conditions physico-chimiques mesurées lors de l'observation des espèces.
Dans l'ensemble des archives Dwc qui sont des fichiers relationnels, le point commun est la colonne **eventID**. La colonne **occurrenceID** permet également d'établir les relations entres les paramètres et les occurrences.

## Transformation des données

Afin d'alléger la lecture de la table finale, supprimons quelques colonnes non utilisées dans les fichiers *event* et *occurrence core*.

In [3]:
event <- event_raw %>% select(eventID,eventDate, decimalLongitude, decimalLatitude)
occurrence <- occurrence_raw %>% select(eventID, occurrenceID, kingdom, ScientificName)


On peut désormais grouper ses deux fichiers ensemble :

In [4]:
data_core <- merge(event,occurrence, by='eventID')
str(data_core)


'data.frame':	1667 obs. of  7 variables:
 $ eventID         : chr  "CZIP-RNE-PF-2019-B1-22" "CZIP-RNE-PF-2019-B2-16" "CZIP-RNE-PF-2019-B2-16" "CZIP-RNE-PF-2019-B2-20" ...
 $ eventDate       : chr  "2019-08-28T09:42" "2019-08-27T17:52" "2019-08-27T17:52" "2019-08-28T08:21" ...
 $ decimalLongitude: num  -69.1 -69.1 -69.1 -69.1 -69.1 ...
 $ decimalLatitude : num  48.6 48.6 48.6 48.6 48.6 ...
 $ occurrenceID    : chr  "CZIP-RNE-PF-2019-ICHTYO-B1-OCC-34" "CZIP-RNE-PF-2019-ICHTYO-B2-OCC-24" "CZIP-RNE-PF-2019-ICHTYO-B2-OCC-25" "CZIP-RNE-PF-2019-ICHTYO-B2-OCC-31" ...
 $ kingdom         : chr  "Animalia" "Animalia" "Animalia" "Animalia" ...
 $ ScientificName  : chr  "Pungitius pungitius" "Gasterosteus aculeatus" "Pungitius pungitius" "Gasterosteus aculeatus" ...


Ensuite on s'intéresse aux fichiers *emof* afin d'extraire les variables d'intérêt. 
Ces fichiers nécessitent une transformation supplémentaire. Si on observe les fichiers *emof* : 

In [5]:
head(emof_occurrence_raw)

eventID                occurrenceID                     
1 CZIP-RNE-PF-2019-B1-22 CZIP-RNE-PF-2019-ICHTYO-B1-OCC-34
2 CZIP-RNE-PF-2019-B1-22 CZIP-RNE-PF-2019-ICHTYO-B1-OCC-34
3 CZIP-RNE-PF-2019-B2-16 CZIP-RNE-PF-2019-ICHTYO-B2-OCC-24
4 CZIP-RNE-PF-2019-B2-16 CZIP-RNE-PF-2019-ICHTYO-B2-OCC-25
5 CZIP-RNE-PF-2019-B2-16 CZIP-RNE-PF-2019-ICHTYO-B2-OCC-25
6 CZIP-RNE-PF-2019-B2-16 CZIP-RNE-PF-2019-ICHTYO-B2-OCC-25
  measurementID             materialSampleID measurementType  measurementValue
1 CZIP-RNE-2019-COUNT-B1:34                  Individual count 1               
2 CZIP-RNE-2019-LT-B1:600   B1:600           Longueur         24              
3 CZIP-RNE-2019-COUNT-B2:24                  Individual count 10              
4 CZIP-RNE-2019-COUNT-B2:25                  Individual count 327             
5 CZIP-RNE-2019-LT-B2:421   B2:421           Longueur         36              
6 CZIP-RNE-2019-LT-B2:422   B2:422           Longueur         42              
  measurementUnit X
1                  
2 mm               
3                  
4                  
5 mm               
6 mm

On remarque que la colonne ***MeasurementType*** contient plusieurs variables. 
On veut obtenir le nombre d'individus du fichier *emof_occurrence* et la salinité, l'oxygène dissout (%) et le pH présent dans le fichier *emof_event*.

In [6]:
emof_occurrence <- emof_occurrence_raw%>%
    select(eventID, occurrenceID, measurementType, measurementValue)%>%
    filter(measurementType == 'Individual count') %>%
    spread(measurementType, measurementValue)
head(emof_occurrence)

emof_event <- emof_event_raw%>%
    select(eventID,measurementType, measurementValue, measurementUnit)%>%
    filter(measurementType %in% c("Oxygène dissout", "Salinité", "pH"), measurementUnit %in% c("psu", "", "%")) %>%
    select(-c(measurementUnit)) %>%
    group_by(eventID) %>%
    spread(measurementType, measurementValue)
head(emof_event)


eventID                occurrenceID                      Individual count
1 CZIP-RNE-PF-2019-B1-22 CZIP-RNE-PF-2019-ICHTYO-B1-OCC-34 1               
2 CZIP-RNE-PF-2019-B2-16 CZIP-RNE-PF-2019-ICHTYO-B2-OCC-24 10              
3 CZIP-RNE-PF-2019-B2-16 CZIP-RNE-PF-2019-ICHTYO-B2-OCC-25 327             
4 CZIP-RNE-PF-2019-B2-20 CZIP-RNE-PF-2019-ICHTYO-B2-OCC-31 2               
5 CZIP-RNE-PF-2019-B2-20 CZIP-RNE-PF-2019-ICHTYO-B2-OCC-32 91              
6 CZIP-RNE-PF-2019-B2-26 CZIP-RNE-PF-2019-ICHTYO-B2-OCC-35 24

eventID                Oxygène dissout pH   Salinité
1 CZIP-RNE-PF-2019-B1-1  99.3            8.51 25.6    
2 CZIP-RNE-PF-2019-B1-15 116.4           8.06 26.02   
3 CZIP-RNE-PF-2019-B1-22 79.6            7.92 26.79   
4 CZIP-RNE-PF-2019-B1-25 111             8.33 26.7    
5 CZIP-RNE-PF-2019-B1-34 73.3            7.2  25.65   
6 CZIP-RNE-PF-2019-B1-39 118.2           8.05 26.95

A partir de là on peut désormais combiner l'ensemble des fichiers, les colonne ***eventID*** et ***occurrenceID*** faisant office de 'clé'.

In [7]:
emof <- merge(emof_event, emof_occurrence, by='eventID')
head(emof)

eventID                Oxygène dissout pH   Salinité
1 CZIP-RNE-PF-2019-B1-22 79.6            7.92 26.79   
2 CZIP-RNE-PF-2019-B2-16 97.7            7.2  23.41   
3 CZIP-RNE-PF-2019-B2-16 97.7            7.2  23.41   
4 CZIP-RNE-PF-2019-B2-20 96.9            7.15 25.5    
5 CZIP-RNE-PF-2019-B2-20 96.9            7.15 25.5    
6 CZIP-RNE-PF-2019-B2-26 95.3            7.29 24.77   
  occurrenceID                      Individual count
1 CZIP-RNE-PF-2019-ICHTYO-B1-OCC-34 1               
2 CZIP-RNE-PF-2019-ICHTYO-B2-OCC-24 10              
3 CZIP-RNE-PF-2019-ICHTYO-B2-OCC-25 327             
4 CZIP-RNE-PF-2019-ICHTYO-B2-OCC-31 2               
5 CZIP-RNE-PF-2019-ICHTYO-B2-OCC-32 91              
6 CZIP-RNE-PF-2019-ICHTYO-B2-OCC-36 236

In [8]:
data_core_emof <- merge(data_core, emof, by=c('eventID', 'occurrenceID'))
head(data_core_emof)



eventID                occurrenceID                      eventDate       
1 CZIP-RNE-PF-2019-B1-22 CZIP-RNE-PF-2019-ICHTYO-B1-OCC-34 2019-08-28T09:42
2 CZIP-RNE-PF-2019-B2-16 CZIP-RNE-PF-2019-ICHTYO-B2-OCC-24 2019-08-27T17:52
3 CZIP-RNE-PF-2019-B2-16 CZIP-RNE-PF-2019-ICHTYO-B2-OCC-25 2019-08-27T17:52
4 CZIP-RNE-PF-2019-B2-20 CZIP-RNE-PF-2019-ICHTYO-B2-OCC-31 2019-08-28T08:21
5 CZIP-RNE-PF-2019-B2-20 CZIP-RNE-PF-2019-ICHTYO-B2-OCC-32 2019-08-28T08:21
6 CZIP-RNE-PF-2019-B2-26 CZIP-RNE-PF-2019-ICHTYO-B2-OCC-35 2019-08-28T16:55
  decimalLongitude decimalLatitude kingdom  ScientificName        
1 -69.09246        48.64362        Animalia Pungitius pungitius   
2 -69.08810        48.64101        Animalia Gasterosteus aculeatus
3 -69.08810        48.64101        Animalia Pungitius pungitius   
4 -69.08810        48.64101        Animalia Gasterosteus aculeatus
5 -69.08810        48.64101        Animalia Pungitius pungitius   
6 -69.08810        48.64101        Animalia Gasterosteus aculeatus
  Oxygène dissout pH   Salinité Individual count
1 79.6            7.92 26.79    1               
2 97.7            7.2  23.41    10              
3 97.7            7.2  23.41    327             
4 96.9            7.15 25.5     2               
5 96.9            7.15 25.5     91              
6 95.3            7.29 24.77    24

On obtient ainsi un fichier ordonné comprenant les mesures d'occurrence et également les mesures associées soit à l'événement soit à l'occurrence. 